In [1]:
#importing all the packages 
import numpy as np
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})
plt.rcParams.update({'font.size': 22})

import pandas as pd
from sklearn import datasets, linear_model
from sklearn.datasets import load_iris
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

C:\Users\grego\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'keras.wrappers'

In [ ]:
# first we will try a single layer perceptron 
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = (iris.target == 0)  # Iris setosa

X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

per_clf = Perceptron(random_state=42)
per_clf.fit(x_train_scaled, y_train)


In [ ]:
print("training set score is : {:.3f}" .format(per_clf.score(x_train_scaled,y_train)))
print("test set score is : {:.3f}" .format(per_clf.score(x_test_scaled,y_test)))


#### Remember the Harvard Clean Energy Project Database that we analyzed in the beginning of the quarter?? 

If not, please look back at Lecture 4 notes 

In [ ]:
# reading a csv file using pandas
data = pd.read_csv('http://faculty.washington.edu/dacb/HCEPDB_moldata.zip')

In [ ]:
df_smaller = data.sample(frac=0.01, random_state=42)

In [ ]:
df_smaller.describe()

In [ ]:
#Separate out the input features from the output target `'pce'`
X = df_smaller[['mass', 'voc', 'jsc', 'e_homo_alpha', 'e_gap_alpha', 
        'e_lumo_alpha']].values
Y = df_smaller[['pce']].values

In [ ]:
X_train_pn, X_test_pn, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.20,
                                                    random_state=42)

In [ ]:
# Create the scaler from the training data only and keep it for later use
X_train_scaler = StandardScaler().fit(X_train_pn)

# Apply the scaler transform to the training data
X_train = X_train_scaler.transform(X_train_pn)

In [ ]:
#visualizing test data before scaling
fig, ax = plt.subplots(1,2,figsize=(10,4))
ax[0].hist(X_test_pn[:,0], alpha=0.6, color='mediumvioletred')
ax[0].set_xlabel('mass')
ax[0].set_ylabel('count')
ax[0].set_title('mass distribution',fontsize=18)
ax[1].hist(X_test_pn[:,1], alpha=0.6, color='royalblue')
ax[1].set_xlabel('voc')
ax[1].set_ylabel('count')
ax[1].set_title('voc distribution',fontsize=18)
plt.tight_layout()
plt.show()

In [ ]:
X_test = X_train_scaler.transform(X_test_pn)

In [ ]:
#visualizing test data after scaling
fig, ax = plt.subplots(1,2,figsize=(10,4))
ax[0].hist(X_test[:,0], alpha=0.6, color='mediumvioletred')
ax[0].set_xlabel('mass')
ax[0].set_ylabel('count')
ax[0].set_title('mass distribution',fontsize=18)
ax[1].hist(X_test[:,1], alpha=0.6, color='royalblue')
ax[1].set_xlabel('voc')
ax[1].set_ylabel('count')
ax[1].set_title('voc distribution',fontsize=18)
plt.tight_layout()
plt.show()

## 2. Creating the neural network model

This is a simple neural network with no hidden layer and one output layer. Here we will use `Keras` functions [Keras documentation](https://keras.io/guides/). We will use `Dense` layers as defined [here](https://keras.io/api/layers/core_layers/dense/), and the `Adam` [optimizer](https://keras.io/api/optimizers/adam/) which relies on gradient descent.

In [ ]:
def simple_model():
    # assemble the structure
    model = Sequential()
    model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
# initialize the andom seed as this is used to generate
# the starting weights
np.random.seed(42)
# create the NN framework
model = simple_model()
history = model.fit(X_train, y_train, validation_split=0.33, epochs=150, 
        batch_size=10000, verbose=0)

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()